In [1]:
# https://fairbank.fas.harvard.edu/?post_types=publication

# if 403 error so first try with requests an put header in it
# then go for cookies(in 503) error

In [11]:
import requests
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
r = requests.get('https://fairbank.fas.harvard.edu/?post_types=publication',headers = headers).text

In [14]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'fairbank_fas_harvard_edu_publication'
c = Crawl()  # creating obje_
# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)
# creating pdf directories
pdf_directory = c.create_pdf_directories(project_path, site)
# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_download_pdf = 0
publish_source = 'fairbank.fas.harvard.edu'
country = 'China'
language = 'English'
images_path = []
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
home_page = requests.get('https://fairbank.fas.harvard.edu/?post_types=publication',headers = headers ,timeout=60).text

for i in home_page.split('<h2 class="entry-title">')[1:2]:
    # source_link
    source_link = c.scrap('<a\s*href="(.*?)"', i)   
    print(source_link)
#     continue
    # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue

    time.sleep(random.randint(1,3))

    logger.info(f'Fetching {source_link}\n')

    page = c.download_page(source_link)  
    print(page)
    continue
    if page.startswith('Unable to fetch'):
        logger.info(page)
        unable_to_fetch_article_url += 1
        continue    

    source_headline = c.scrap('<title>(.*?)\|', page)

    # skip if headline not found
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue

     # Date and time
    pub_date, publish_time = '', current_time

    try:   
        date_time_str = c.scrap('"published">(.*?)<', page)
        date_time_str = re.sub('&.*?;','' , date_time_str)
        date_time_str = re.sub('[^\w+]', '', date_time_str)  
        date_time_obj = datetime.strptime(date_time_str, '%b%d%Y')#2021-04-09T00:00:00
        ist_date_time = date_time_obj - timedelta(hours = 0,minutes = 0)  
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]
    
    except:
        pass

    # skip null date
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')            
        skipped_due_to_date += 1
        continue

    # break if date is not today's date
#         if pub_date != created_on:
#             break    

    # source_content          
    source_content = c.scrap('<strong>(.*?)</p>\s*<p><a\s*href',page)    
    source_content = re.sub('<span\s*style=(.*?)</span>','',source_content,re.S)
    source_content = re.sub('&.*?;','',source_content,re.S)
    source_content = c.strip_html(source_content)
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')            
        skipped_due_to_content += 1
        continue


    journalist =c.scrap("'author-name':'(.*?)'",page)

    if not journalist: journalist = 'NA'

    # current date and time 00
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []
    
    pdf_path, pdf_name = '', ''
    pdf_url = c.scrap('<meta\s*name="citation_pdf_url".*?content="(.*?)"',page)
 
     #https://www.adb.org/sites/default/files/publication/696296/adbi-wp1251.pdf
    pdf_name = c.scrap('.*\/(.*)',pdf_url)
    # pdf_path
    pdf_path = f'{pdf_directory}/{pdf_name}'        

    # download pdf
    pdf = c.download_pdf(pdf_url, pdf_path)
    if pdf.startswith('Unable to fetch'):
        logger.info(pdf) # writes error message with error code
        unable_to_download_pdf += 1
        continue
  
    # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":'',
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content,
                    "pdf_url": pdf_url,
                    "pdf_name": pdf_name,
                    "pdf_path":pdf_path
                }

#     cl_data.insert_one(clientdata)  
    no_of_data += 1
logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Unable to download pdf: {unable_to_download_pdf}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n') 

https://fairbank.fas.harvard.edu/publication/agricultural-development-in-china-1368-1968/
<!DOCTYPE html> <!--[if IE 6]> <html id="ie6" lang="en-US"> <![endif]--> <!--[if IE 7]> <html id="ie7" lang="en-US"> <![endif]--> <!--[if IE 8]> <html id="ie8" lang="en-US"> <![endif]--> <!--[if !(IE 6) | !(IE 7) | !(IE 8)  ]><!--> <html lang="en-US"> <!--<![endif]--> <head> 	<meta charset="UTF-8" /> 			 	<meta http-equiv="X-UA-Compatible" content="IE=edge"> 	<link rel="pingback" href="https://fairbank.fas.harvard.edu/xmlrpc.php" />  		<!--[if lt IE 9]> 	<script src="https://fairbank.fas.harvard.edu/wp-content/themes/Divi/js/html5.js" type="text/javascript"></script> 	<![endif]-->  	<script type="text/javascript">
		document.documentElement.className = 'js';
	</script>  	<script>var et_site_url='https://fairbank.fas.harvard.edu';var et_post_id='3374';function et_core_page_resource_fallback(a,b){"undefined"===typeof b&&(b=a.sheet.cssRules&&0===a.sheet.cssRules.length);b&&(a.onerror=null,a.onload=nu